Hello world! This is my first colab notebook to run prediction models on big data.

# 1. load data from cloud storage

In [0]:
from google.colab import auth
auth.authenticate_user()

# Download a file from GCS

### Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

### Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

In [5]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

# Create the service client.
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

from apiclient.http import MediaIoBaseDownload

with open('/tmp/application_train.csv', 'wb') as f:
  request = gcs_service.objects().get_media(bucket='chrisquared',
                                            object='home_credit_default_risk/application_train.csv')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


In [0]:
import pandas as pd
import numpy as np
import sklearn
import random
import datetime

import matplotlib.pyplot as plt
% matplotlib inline
from matplotlib import mlab

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [0]:
application_train = pd.read_csv("/tmp/application_train.csv")

In [9]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### defining some useful functions

In [0]:
def train_test_fn(df , proportion):

    msk = np.random.rand(len(df)) < proportion

    train = df[msk]

    test = df[~msk]
    
    return train, test

def confusion_matrix_fn(truth,result):
    c11 = sum(truth*result)
    c10 = sum(truth*(1-result))
    c01 = sum((1-truth)*result)
    c00 = sum((1-truth)*(1-result))
    tab = np.matrix([[c11, c10], [c01, c00]])
    return tab

def precision_fn(truth,result):
    c11 = sum(truth*result)
    #c10 = sum(truth*(1-result))
    c01 = sum((1-truth)*result)
    #c00 = sum((1-truth)*(1-result))
    prec = c11/(c11+c01)
    return prec

def recall_fn(truth,result):
    c11 = sum(truth*result)
    c10 = sum(truth*(1-result))
    #c01 = sum((1-truth)*result)
    #c00 = sum((1-truth)*(1-result))
    rec = c11/(c11+c10)
    return rec

def F_score_fn(truth,result):
    c11 = sum(truth*result)
    c10 = sum(truth*(1-result))
    c01 = sum((1-truth)*result)
    prec = c11/(c11+c01)
    rec = c11/(c11+c10)
    F=2/(1/prec+1/rec)
    
    return F


# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

### Running a first random forest classifier prediction based on the dataset "application_train.csv"

In [0]:
application_train.fillna('missing', inplace = True)

In [0]:
application_train = pd.get_dummies(application_train)

In [0]:
train, test = train_test_fn(application_train, 0.6)

In [0]:
X_train = train
y_train = train.pop('TARGET')

X_test = test
y_test = test.pop('TARGET')

In [0]:
clf = RandomForestClassifier(n_estimators=1000 , max_depth=10, random_state=0)
clf.fit(X, y)

In [0]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            #min_impurity_decrease=0.0, #min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [0]:
precision_fn(clf.predict(X_test),y_test)

In [0]:
confusion_matrix_fn(clf.predict(X_test),y_test)